In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from pandas import Series, DataFrame
from collections import Counter, defaultdict
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import KBinsDiscretizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from numpy.random import RandomState
#Загружаем данные
data = pd.read_excel('credit_train.xlsx')

In [2]:
data.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,59998.00,10,2019-06-01 00:00:00,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,10889.00,6,2019-01-01 00:00:00,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,10728.00,12,2019-01-01 00:00:00,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,12009.09,12,2019-01-01 00:00:00,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,2019-01-01 00:00:00,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


In [3]:
data_object = data.select_dtypes(include = 'object')
data_object

,gender,marital_status,job_position,tariff_id,education,living_region
0,M,NaN,UMN,2019-06-01 00:00:00,GRD,КРАСНОДАРСКИЙ КРАЙ
1,F,MAR,UMN,2019-01-01 00:00:00,NaN,МОСКВА
2,M,MAR,SPC,2019-01-01 00:00:00,NaN,ОБЛ САРАТОВСКАЯ
3,F,NaN,SPC,2019-01-01 00:00:00,NaN,ОБЛ ВОЛГОГРАДСКАЯ
4,M,NaN,SPC,2019-01-01 00:00:00,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ
...,...,...,...,...,...,...
170741,F,UNM,SPC,2019-01-01 00:00:00,GRD,РЕСПУБЛИКА ТАТАРСТАН
170742,F,MAR,SPC,2019-06-01 00:00:00,SCH,САНКТ-ПЕТЕРБУРГ Г
170743,F,UNM,SPC,1932-01-01 00:00:00,GRD,ПРИМОРСКИЙ КРАЙ
170744,F,DIV,PNA,2019-01-01 00:00:00,SCH,ПЕНЗЕНСКАЯ ОБЛ


In [4]:
data_object.describe()

,gender,marital_status,job_position,tariff_id,education,living_region
count,170746,170743,170746,170746,170741,170554
unique,2,5,18,32,5,301
top,F,MAR,SPC,2019-01-01 00:00:00,SCH,ОБЛ МОСКОВСКАЯ
freq,88697,93954,134680,71457,87537,12228


In [5]:
#Посмотрим на наши категориальные переменные и узнаем количество редких категорий в них
for i in data_object.columns:
    print(data_object[i].value_counts(dropna = False))

F    88697
M    82049
Name: gender, dtype: int64
MAR    93954
UNM    52149
DIV    16969
CIV     4196
WID     3475
NaN        3
Name: marital_status, dtype: int64
SPC    134680
UMN     17674
BIS      5591
PNA      4107
DIR      3750
ATP      2791
WRK       656
NOR       537
WOI       352
INP       241
BIU       126
WRP       110
PNI        65
PNV        40
PNS        12
HSK         8
INV         5
ONB         1
Name: job_position, dtype: int64
2019-01-01 00:00:00    71457
2019-06-01 00:00:00    39117
1932-01-01 00:00:00    15537
2019-04-01 00:00:00    10970
2019-05-01 00:00:00     7497
2019-09-01 00:00:00     5538
1943-01-01 00:00:00     3930
2019-03-01 00:00:00     3339
2016-01-01 00:00:00     3232
1.0                     2245
1944-01-01 00:00:00     2228
2019-07-01 00:00:00     1007
2020-01-01 00:00:00      766
2017-01-01 00:00:00      717
2021-01-01 00:00:00      579
2019-02-01 00:00:00      540
1994-01-01 00:00:00      414
2022-01-01 00:00:00      376
2023-01-01 00:00:00      370
19

In [6]:
#Их будем рейргруппить
groups_for_rare = ['job_position', 'tariff_id', 'education']
groups_for_rare

['job_position', 'tariff_id', 'education']

In [7]:
#Заменяем пропущенные значения в marital_status, education и living_region
data['marital_status'] = np.where(data['marital_status'].isnull(), 'MAR', data['marital_status'])
data['education'] = np.where(data['education'].isnull(), 'SCH', data['education'])
data['living_region'] = np.where(data['living_region'].isnull(), 'ОБЛ МОСКОВСКАЯ', data['living_region'])

In [8]:
#Выделяем количественные признаки
data_num = data.select_dtypes(include = ['float64', 'int64'])
data_num.head()

,client_id,age,credit_sum,credit_month,score_shk,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,NaN,59998.00,10,NaN,30000.0,1.0,1.0,0
1,2,NaN,10889.00,6,NaN,NaN,2.0,0.0,0
2,3,32.0,10728.00,12,NaN,NaN,5.0,0.0,0
3,4,27.0,12009.09,12,NaN,NaN,2.0,0.0,0
4,5,45.0,NaN,10,0.421385,NaN,1.0,0.0,0


In [9]:
#Посмотрим на наши категориальные переменные и узнаем количество редких категорий в них
for i in data_num.columns:
    print(data_num[i].value_counts(dropna = False))

2047     1
46396    1
34106    1
40249    1
38200    1
        ..
35524    1
45763    1
47810    1
41665    1
2049     1
Name: client_id, Length: 170746, dtype: int64
28.0    7591
29.0    7528
30.0    7328
27.0    7086
26.0    6823
31.0    6768
32.0    6633
33.0    6576
25.0    6151
34.0    5877
35.0    5650
24.0    5430
36.0    5405
37.0    5259
38.0    5125
39.0    5009
40.0    4932
41.0    4610
42.0    4370
23.0    4294
43.0    4073
44.0    3776
22.0    3531
46.0    3237
45.0    3220
47.0    2871
48.0    2626
49.0    2368
50.0    2367
21.0    2356
52.0    1988
51.0    1968
53.0    1932
55.0    1805
54.0    1793
56.0    1530
57.0    1414
58.0    1232
59.0    1065
20.0    1059
60.0     958
61.0     837
62.0     769
63.0     640
64.0     560
65.0     465
19.0     430
66.0     364
67.0     362
68.0     241
69.0     181
70.0     142
18.0     106
71.0      32
NaN        3
Name: age, dtype: int64
22990.00    618
53189.00    521
39990.00    461
57789.00    452
23990.00    433
           ...

In [10]:
#Заменяем пропущенные значения в age, score_shk, monthly_income и credit_sum 
data['age'] = np.where(data['age'].isnull(), data['age'].median(), data['age'])
data['score_shk'] = np.where(data['score_shk'].isnull(), data['score_shk'].median(), data['score_shk'])
data['credit_sum'] = np.where(data['credit_sum'].isnull(), data['credit_sum'].median(), data['credit_sum'])
data['monthly_income'] = np.where(data['monthly_income'].isnull(), data['monthly_income'].median(), data['monthly_income'])

#Конструирование признаков

In [11]:
#Первый созданный нами признак - Payment - сумма кредита / срок кредита
data['payment'] = data['credit_sum'] / data['credit_month']

In [12]:
#Второй признак - share_income - доля дохода от суммы кредита
data['share_income'] = data['monthly_income'] / data['credit_sum']

In [13]:
#Третий признак - income_to_credit - отношение месячной зарплаты к временному периоду погашения кредита
data['income_to_credit'] = data['age'] / data['credit_month']

In [14]:
data['payment_by_score'] = data['payment'] * data['score_shk']

In [15]:
data['payment_to_age'] = data['payment'] / data['age']

In [16]:
data['score_shk_by_credit_sum'] = data['score_shk'] * data['credit_sum']


In [17]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('open_account_flg', axis=1), 
                                                data['open_account_flg'], 
                                                test_size=.3, 
                                                stratify=data['open_account_flg'], 
                                                random_state=100)

In [18]:
X_train.drop('client_id', axis=1, inplace=True)

#Работа с регионами

In [19]:
X_train['living_region'] = np.where((X_train['living_region'] == 'КРАЙ КРАСНОДАРСКИЙ')
                                 |(X_train['living_region'] == 'КРАСНОДАРСКИЙ'), 'КРАСНОДАРСКИЙ КРАЙ', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'МОСКВА Г')
                                 |(X_train['living_region'] == 'Г МОСКВА')
                                 |(X_train['living_region'] == 'Г.МОСКВА')
                                 |(X_train['living_region'] == 'Г. МОСКВА'), 'МОСКВА', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'ОБЛ.МОСКОВСКАЯ')
                                 |(X_train['living_region'] == 'МОСКОВСКАЯ ОБЛ')
                                 |(X_train['living_region'] == 'МОСКОВСКАЯ')
                                 |(X_train['living_region'] == 'МОСКВОСКАЯ ОБЛ')
                                 |(X_train['living_region'] == 'МОСКОВСКАЯ ОБЛАСТЬ'), 'ОБЛ МОСКОВСКАЯ', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'САНКТ-ПЕТЕРБУРГ Г')
                                 |(X_train['living_region'] == 'Г. САНКТ-ПЕТЕРБУРГ'), 'САНКТ-ПЕТЕРБУРГ', X_train['living_region'])


X_train['living_region'] = np.where((X_train['living_region'] == 'РЕСП ТАТАРСТАН')
                                 |(X_train['living_region'] == 'РЕСПУБЛИКАТАТАРСТАН')
                                 |(X_train['living_region'] == 'РЕСПУБЛИКА ТАТАРСТАН'), 'ТАТАРСТАН РЕСП', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'РЕСП.БАШКОРТОСТАН')
                                 |(X_train['living_region'] == 'РЕСП. БАШКОРТОСТАН')
                                 |(X_train['living_region'] == 'БАШКОРТОСТАН')
                                 |(X_train['living_region'] == 'БАШКОРТОСТАН РЕСП'), 'РЕСП БАШКОРТОСТАН', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'ИРКУТСКАЯ ОБЛ')
                                 |(X_train['living_region'] == 'ИРКУТСКАЯ ОБЛАСТЬ'), 'ОБЛ ИРКУТСКАЯ', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'ОБЛ СВЕРДЛОВСКАЯ')
                                 |(X_train['living_region'] == 'ОБЛ. СВЕРДЛОВСКАЯ')
                                 |(X_train['living_region'] == 'СВЕРДЛОВСКАЯ')
                                 |(X_train['living_region'] == 'СВЕРДЛОВСКАЯ ОБЛАСТЬ'), 'СВЕРДЛОВСКАЯ ОБЛ', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'ОБЛ.НИЖЕГОРОДСКАЯ')
                                 |(X_train['living_region'] == 'НИЖЕГОРОДСКАЯ ОБЛ'), 'ОБЛ НИЖЕГОРОДСКАЯ', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'САХА /ЯКУТИЯ/')
                                 |(X_train['living_region'] == 'САХА (ЯКУТИЯ)'), 'САХА', X_train['living_region'])

X_train['living_region'] = np.where((X_train['living_region'] == 'ЧУВАШСКАЯ  - ЧУВАШИЯ')
                                 |(X_train['living_region'] == 'ЧУВАШСКАЯ - ЧУВАШИЯ')
                                 |(X_train['living_region'] == 'ЧУВАШИЯ ЧУВАШСКАЯ  -')
                                 |(X_train['living_region'] == 'ЧУВАШСКАЯ'), 'ЧУВАШИЯ', X_train['living_region'])



In [20]:
X_train['living_region'] = X_train['living_region'].str.replace('КРАЙ', '')
X_train['living_region'] = X_train['living_region'].str.replace('РЕСПУБЛИКА', '')
X_train['living_region'] = X_train['living_region'].str.replace('РЕСП.', '')
X_train['living_region'] = X_train['living_region'].str.replace('РЕСП', '')
X_train['living_region'] = X_train['living_region'].str.replace('ОБЛАСТЬ', '')
X_train['living_region'] = X_train['living_region'].str.replace(' ОБЛАСТЬ', '')
X_train['living_region'] = X_train['living_region'].str.replace('ОБЛ ', '')
X_train['living_region'] = X_train['living_region'].str.replace(' ОБЛ', '')
X_train['living_region'] = X_train['living_region'].str.replace('ОБЛ. ', '')
X_train['living_region'] = X_train['living_region'].str.replace('.', '')
X_train['living_region'] = X_train['living_region'].str.strip(' ')
X_train['living_region'] = X_train['living_region'].str.replace(' АО', '')
X_train['living_region'] = X_train['living_region'].str.replace('АО ', '')
X_train['living_region'] = X_train['living_region'].str.replace('АО', '')
X_train['living_region'] = X_train['living_region'].str.strip(' ')

In [21]:
X_train['living_region'] = np.where((X_train['living_region'].isnull())|(X_train['living_region'] == 'РОССИЯ')|(X_train['living_region'] == 'ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ'), 'Центральный', X_train['living_region'])

In [22]:
X_train['living_region'].isnull().sum()

0

In [23]:
X_train['living_region'].value_counts()

МОСКОВСКАЯ              9134
МОСКВА                  6417
САНКТ-ПЕТЕРБУРГ         5844
КРАСНОДАРСКИЙ           5817
ТАТАРСТАН               5394
                        ... 
ЭВЕНКИЙСКИЙ                1
АЕВРЕЙСКАЯ                 1
ДАЛЬНИЙ ВОСТОК             1
ОБЛСАРАТОВСКАЯ             1
ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н       1
Name: living_region, Length: 105, dtype: int64

In [24]:
#Создаем функцию для проверки признаков по AUC
def importace_auc(train, test):
    train_copy = train.copy()
    test_copy = test.copy()
    #Выбираем нужные нам признаки с числовым типом данных
    col_list = train.select_dtypes(include = ['number']).columns
    #Создаем словарь
    auc_list = []
    for i in col_list:
        #Заменяем все пропущенные значения на медианное
        train_copy[i].fillna(train_copy[i].median(), inplace = True)
        # Создаем экземляр класса PowerTransformer
        power = PowerTransformer(method = 'yeo-johnson', standardize = True).fit(train_copy[[i]])
        #Трансформируем данные тренировочной выборки
        train_copy[i] = power.transform(train_copy[[i]])
        #Заменяем все пропущенные значения на медианное
        test_copy[i].fillna(test_copy[i].median(), inplace = True)
        #Трансформируем данные тестовой выборки
        test_copy[i] = power.transform(test_copy[[i]])
        #Создаем экземпляр класса LogisticRegression
        logreg = LogisticRegression(solver = 'liblinear').fit(train_copy[[i]], y_train)
        #Подсчитываем значение AUC для каждой категории
        auc = roc_auc_score(y_test, logreg.predict_proba(test_copy[[i]])[:, 1])
        #Добавляет в наш пустой список значения auc
        auc_list.append(auc)
    #Создаем датафрейм с показателями auc
    result = pd.DataFrame({'Переменная': col_list, 'AUC': auc_list})
    #Округляем значения и сортируем по убыванию
    result = np.round(result.sort_values(by = 'AUC', ascending = False), 3)
    #Форматирование по цвету
    cm = sns.light_palette('yellow', as_cmap = True)
    return(result.style.background_gradient(cmap = cm))

In [25]:
#запускаем фунцию
importace_auc(X_train, X_test)

C:\Users\Embulatov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


,Переменная,AUC
7,payment,0.562
1,credit_sum,0.554
10,payment_by_score,0.542
8,share_income,0.538
11,payment_to_age,0.538
12,score_shk_by_credit_sum,0.536
3,score_shk,0.535
0,age,0.534
9,income_to_credit,0.53
2,credit_month,0.529


In [26]:
def diagnostics_skewness(df):
    # создаем список переменных
    col_list = df.select_dtypes(include=['number']).columns
    skew_initial_list = []
    skew_pos_reciprocal_list = []
    skew_neg_reciprocal_list = []
    skew_log_list = []
    skew_corr_log_001_list = []
    skew_corr_log_01_list = []
    skew_corr_log_1_list = []
    skew_corr_log_5_list = []
    skew_cbrt_list = []
    skew_sqrt_list = []
    
    # создаем копию датафрейма
    df_ = X_train.copy()
    # запускаем цикл, который вычисляет скос по каждой 
    # преобразованной переменной
    for i in col_list:
        df_[i] = df_[i].fillna(df_[i].median())
        skew_initial = df_[i].skew()
        skew_pos_reciprocal = (1 / (df_[i].clip(0.01))).skew()
        skew_neg_reciprocal = (-1 / (df_[i].clip(0.01))).skew()
        skew_log = np.log(df[i].clip(0.01)).skew()
        skew_corr_log_001 = np.log((df_[i].clip(0.01) / df_[i].mean()) + 0.001).skew()
        skew_corr_log_01 = np.log((df_[i].clip(0.01) / df_[i].mean()) + 0.01).skew()
        skew_corr_log_1 = np.log((df_[i].clip(0.01) / df_[i].mean()) + 0.1).skew()
        skew_corr_log_5 = np.log((df_[i].clip(0.01) / df_[i].mean()) + 0.5).skew()
        skew_cbrt = np.cbrt(df_[i].abs()).skew()
        skew_sqrt = np.sqrt(df_[i].abs()).skew()
        skew_initial_list.append(skew_initial)
        skew_pos_reciprocal_list.append(skew_pos_reciprocal)
        skew_neg_reciprocal_list.append(skew_neg_reciprocal)
        skew_log_list.append(skew_log)
        skew_corr_log_001_list.append(skew_corr_log_001)
        skew_corr_log_01_list.append(skew_corr_log_01)
        skew_corr_log_1_list.append(skew_corr_log_1)
        skew_corr_log_5_list.append(skew_corr_log_5)
        skew_cbrt_list.append(skew_cbrt)
        skew_sqrt_list.append(skew_sqrt)   

    result = pd.DataFrame({'Переменная': col_list,
                           'Skew_init': skew_initial_list,
                           'Skew_pos_recip': skew_pos_reciprocal_list,
                           'Skew_neg_recip': skew_neg_reciprocal_list,
                           'Skew_log': skew_log_list,
                           'Skew_adj_log (k=0.001)': skew_corr_log_001_list,
                           'Skew_adj_log (k=0.01)': skew_corr_log_01_list,
                           'Skew_adj_log (k=0.1)': skew_corr_log_1_list,
                           'Skew_adj_log (k=0.5)': skew_corr_log_5_list,
                           'Skew_cbrt': skew_cbrt_list,
                           'Skew_sqrt': skew_sqrt_list})
    result = np.round(result.sort_values(by='Skew_init', ascending=False), 3)
    cm = sns.light_palette('magenta', as_cmap=True)
    return(result.style.background_gradient(cmap=cm))

In [27]:
diagnostics_skewness(X_train)

,Переменная,Skew_init,Skew_pos_recip,Skew_neg_recip,Skew_log,Skew_adj_log (k=0.001),Skew_adj_log (k=0.01),Skew_adj_log (k=0.1),Skew_adj_log (k=0.5),Skew_cbrt,Skew_sqrt
8,share_income,5.387,2.331,-2.331,0.307,0.309,0.331,0.509,0.968,1.074,1.596
4,monthly_income,5.309,1.552,-1.552,0.266,0.268,0.287,0.446,0.886,1.008,1.531
6,overdue_credit_count,4.601,-4.475,4.475,4.335,4.476,4.476,4.477,4.478,4.48,4.491
2,credit_month,2.67,3.443,-3.443,-0.226,-0.222,-0.188,0.102,0.837,0.817,1.312
9,income_to_credit,2.56,2.019,-2.019,0.132,0.134,0.154,0.321,0.754,0.794,1.161
11,payment_to_age,2.274,2.809,-2.809,0.043,0.046,0.072,0.266,0.67,0.651,0.962
10,payment_by_score,1.971,345.714,-345.714,0.052,0.093,0.125,0.315,0.711,0.692,0.984
7,payment,1.912,2.408,-2.408,0.116,0.119,0.14,0.302,0.66,0.648,0.911
12,score_shk_by_credit_sum,1.684,345.719,-345.719,-0.182,-0.116,-0.078,0.147,0.57,0.517,0.807
1,credit_sum,1.442,2.323,-2.323,-0.068,-0.066,-0.043,0.134,0.501,0.458,0.701


In [28]:
#share_income - Skew_log
#monthly_income - Skew_log
#credit_month - Skew_adj_log (k=0.1)
#income_to_credit - Skew_log
#payment_to_age - Skew_log
#payment_by_score - Skew_log
#payment - Skew_adj_log (k=0.01)
#score_shk_by_credit_sum - Skew_adj_log (k=0.01)
#credit_sum - Skew_adj_log (k=0.01)
#credit_count - Skew_sqrt
#age - Skew_log
#score_shk - Skew_sqrt

In [29]:
num_for_skew_log = ['share_income', 'monthly_income', 'income_to_credit', 'payment_to_age', 'payment_by_score', 'age']
num_for_skew_adj_log0_1 = ['credit_month']
num_for_skew_adj_log0_01 = ['score_shk_by_credit_sum', 'credit_sum', 'payment']
num_for_sqrt = ['credit_count', 'score_shk']

In [30]:
all_cat_columns = X_train.select_dtypes(include = 'object').columns

In [31]:
cat_columns = list(set(all_cat_columns).difference(set(groups_for_rare)))

In [32]:
all_non_num = num_for_skew_log + num_for_skew_adj_log0_1 + num_for_skew_adj_log0_01 + num_for_sqrt
all_non_num

['share_income',
 'monthly_income',
 'income_to_credit',
 'payment_to_age',
 'payment_by_score',
 'age',
 'credit_month',
 'score_shk_by_credit_sum',
 'credit_sum',
 'payment',
 'credit_count',
 'score_shk']

In [33]:
all_num_columns = X_train.select_dtypes(include = 'number').columns
all_num_columns

Index(['age', 'credit_sum', 'credit_month', 'score_shk', 'monthly_income',
       'credit_count', 'overdue_credit_count', 'payment', 'share_income',
       'income_to_credit', 'payment_by_score', 'payment_to_age',
       'score_shk_by_credit_sum'],
      dtype='object')

In [34]:
num_columns = list(set(all_num_columns).difference(set(all_non_num)))
num_columns

['overdue_credit_count']

#Создание классов

In [35]:
# создаем собственный класс CorrLog, выполняюший 
# преобразование log(x/mean(x)+k)
class CorrLog(BaseEstimator, TransformerMixin):
    """
    Параметры:
    lower: нижнее пороговое значение
    a: константа
    k: корректировка асимметрии
    """
    def __init__(self, lower=0.001, k=0.2):
        self.lower = lower
        self.k = k
        self.mean = None
        
    def fit(self, X, y=None):
        self.mean = X.mean()
        return self
    
    def transform(self, X):
        X = np.log(X.clip(self.lower) / self.mean + self.k)
        return X

In [36]:
#для укрупнения категорий признака job position мы создадим специальный класс
class raregrouper(BaseEstimator, TransformerMixin):
    def __init__(self, th = 1000):
        self.th = th
    def fit(self, X, y = None):
        cnt = Counter(X)
        self.keeplist = [x for x, y in cnt.most_common() if y > self.th]
        return self
    def transform(self, X):
        X = np.where(~X.isin(self.keeplist), 'Other', X)
        return X

In [37]:
# создаем собственный класс Replacer, заменяющие отрицательные
# и нулевые значения на небольшие положительные (необходимо
# перед применением логарифмического и обратного преобразований)
class Replacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        repl: значение для замены
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [38]:
dicta = {98: 'Other'}
X_train['living_region'].replace(dicta, inplace=True)

In [39]:
X_train['tariff_id'] = np.where(X_train['tariff_id'] == '1.0', '2019-01-01 00:00:00', X_train['tariff_id'])

In [40]:
num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [41]:
num_sqrt_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('sqrt', FunctionTransformer(np.sqrt, validate=False)),
    ('rep', Replacer(repl_value=0.1)),
    ('scaler', StandardScaler())
])

In [42]:
num_log0_01_pipe = Pipeline([
    ('corrlog0_1', CorrLog(k=0.01)),
    ('rep', Replacer(repl_value=0.1)),
    ('scaler', StandardScaler())
])

In [43]:
num_skew_adj_log0_1_pipe = Pipeline([
    ('corrlog0_1', CorrLog(k=0.1)),
    ('rep', Replacer(repl_value=0.1)),
    ('scaler', StandardScaler())
])

In [44]:
num_skew_log_pipe = Pipeline([
    ('rep', Replacer(repl_value=0.1)),
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler())
])

In [45]:
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [46]:
cat_rare_pipe = Pipeline([
    ('rar', raregrouper()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [47]:
transformers = [('num', num_pipe, num_columns),
                ('num_sqrt', num_sqrt_pipe, num_for_sqrt),
                ('num_log_0_01', num_log0_01_pipe, num_for_skew_adj_log0_01),
                ('num_log0_1', num_skew_adj_log0_1_pipe, num_for_skew_adj_log0_1),
                ('num_log', num_skew_log_pipe, num_for_skew_log),
                ('cat', cat_pipe, cat_columns),
                ('cat_rare', cat_rare_pipe, groups_for_rare)]

In [48]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
pipe = Pipeline([('tf', transformer), 
                 ('boost', GradientBoostingClassifier(random_state=42))])

In [50]:
#Параметры для модели
param_grid = {
    'boost__max_depth': [8],
    'boost__learning_rate' : [0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'tf__cat_rare__rar__th': [2000, 2500, 3000]}


In [51]:
#Загрузка экземпляра класса перекрестной проверки
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [52]:
#Модель GridSearchCV
gs = GridSearchCV(pipe, param_grid, scoring='roc_auc',cv=strat, return_train_score=False)

In [53]:
#Фитинг модели
gs.fit(X_train, y_train)
# Наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# Наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(gs.best_score_))
# Значение правильности
print('Значение правильности на тестовой выборке: {:.3f}'.format(gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'boost__learning_rate': 0.05, 'boost__max_depth': 8, 'tf__cat_rare__rar__th': 2000}
Наилучшее значение правильности: 0.682
Значение правильности на тестовой выборке: 0.666


In [54]:
pipe_pipe = Pipeline([('tf', transformer), 
                 ('logreg', LogisticRegression(solver='liblinear', random_state=42))])

In [ ]:
param_grid = {}


In [ ]:
cat_columns_test = X_train.select_dtypes('object').columns
num_columns_test = X_train.select_dtypes(include = ['int64', 'float64']).columns

In [ ]:
#Трансформер для категориальных переменных
cat_pipe_test = Pipeline([
    ('imp', SimpleImputer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [ ]:
#Трансформер для количественных переменных
num_pipe_test = Pipeline([
    ('imp', SimpleImputer()),
    ('scaler', StandardScaler())])

In [ ]:
#Конвеер
transformers_test = [('num', num_pipe_test, num_columns_test),
               ('cat', cat_pipe_test, cat_columns_test)]

In [ ]:
transformer_test = ColumnTransformer(transformers=transformers_test)

In [ ]:
#Финальный трансформер
pipe_test = Pipeline([('tf', transformer_test), 
                 ('logreg', LogisticRegression(solver='liblinear', random_state=42))])

In [ ]:
#Параметры для модели
param_grid_test = {
    'tf__num__imp__strategy': ['mean', 'median', 'constant'],
    'tf__cat__imp__strategy': ['most_frequent', 'constant'],
    'tf__cat__rare'
    'logreg': [GradientBoostingClassifier(n_estimators=50,
                                               random_state=42,
                                               subsample=0.8)]}

In [ ]:
#Модель GridSearchCV
gs_test = GridSearchCV(pipe_test, param_grid_test, scoring='roc_auc',cv=strat, return_train_score=False)

In [ ]:
gs_test.fit(X_train, y_train)

In [ ]:
# Наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs_test.best_params_))
# Наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(gs_test.best_score_))
# Значение правильности
print('Значение правильности на тестовой выборке: {:.3f}'.format(gs_test.score(X_test, y_test)))